<a href="https://colab.research.google.com/github/MEnisSen/indian_pines_cnn_tf/blob/main/HSI_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **DATA**

### **Basic CNN**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def zero_pad(X, pad):
  return np.pad(X, ((0, 0),(pad, pad),(pad, pad),(0, 0)), 'constant', constant_values=0)

In [ ]:
def conv_single_step(a_slice_prev, W, b):
  return np.sum(a_slice_prev * W) + float(b)

In [ ]:
def conv_forward(A_prev, W, b, hyperparameters):
  (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
  (f, f, n_C_prev, n_C) = W.shape
  stride = hyperparameters["stride"]
  pad = hyperparameters["pad"]

  n_H = ((n_H_prev-f+2*pad)/stride)+1
  n_W = ((n_W_prev-f+2*pad)/stride)+1

  Z = np.zeros((int(m), int(n_H), int(n_W), int(n_C)))
  A_prev_pad = zero_pad(A_prev, pad)

  for i in range(m):
    a_prev_pad = A_prev_pad[i]
    for h in range(int(n_H)):
      vert_start = h*stride
      vert_end = vert_start+f
          
      for w in range(int(n_W)):
        horiz_start = w*stride
        horiz_end = horiz_start+f
              
        for c in range(int(n_C)):
          a_slice_prev = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :]
                  
          weights = W[:, :, :, c]
          biases  = b[:, :, :, c]
          Z[i, h, w, c] = conv_single_step(a_slice_prev, weights, biases)

  cache = (A_prev, W, b, hyperparameters)
  return Z, cache

In [ ]:
def pool_forward(A_prev, hyperparameters, mode = 'max'):
  (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
  f = hyperparameters["f"]
  stride = hyperparameters["stride"]

  n_H = int(1 + (n_H_prev - f) / stride)
  n_W = int(1 + (n_W_prev - f) / stride)
  n_C = n_C_prev

  A = np.zeros((m, n_H, n_W, n_C))

  for i in range(m):
    a_prev_slice = A_prev[i]
    for h in range(n_H):
      vert_start = stride * h 
      vert_end = vert_start  + f
            
      for w in range(n_W):
        horiz_start = stride * w
        horiz_end = horiz_start + f
                
        for c in range(n_C):
          a_slice_prev = a_prev_slice[vert_start:vert_end,horiz_start:horiz_end,c]
                    
          if mode == "max":
            A[i, h, w, c] = np.max(a_slice_prev)
          elif mode == "average":
            A[i, h, w, c] = np.mean(a_slice_prev)

  cache = (A_prev, hyperparameters)
  return A, cache

In [ ]:
def conv_backward(dZ, cache):
  (A_prev, W, b, hparameters) = cache
  (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
  (f, f, n_C_prev, n_C) = W.shape
  stride = hparameters["stride"]
  pad = hparameters["pad"]
  (m, n_H, n_W, n_C) = dZ.shape
    
  dA_prev = np.zeros((m, n_H_prev, n_W_prev, n_C_prev))                           
  dW = np.zeros((f, f, n_C_prev, n_C))
  db = np.zeros((1, 1, 1, n_C))

  A_prev_pad = zero_pad(A_prev, pad)
  dA_prev_pad = zero_pad(dA_prev, pad)
  
  for i in range(m):
    a_prev_pad = A_prev_pad[i]
    da_prev_pad = dA_prev_pad[i]
    
    for h in range(n_H):
      for w in range(n_W):
        for c in range(n_C):
          vert_start = h * stride

          vert_end = vert_start + f
          horiz_start = w * stride

          horiz_end = horiz_start + f

          a_slice = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :]

          da_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :] += W[:,:,:,c] * dZ[i, h, w, c]
          dW[:,:,:,c] += a_slice * dZ[i, h, w, c]
          db[:,:,:,c] += dZ[i, h, w, c]

      dA_prev[i, :, :, :] = da_prev_pad[pad:-pad, pad:-pad, :]
    
  return dA_prev, dW, db

In [ ]:
def create_mask_from_window(x):
  return x == np.max(x)

In [ ]:
def distribute_value(dz, shape):
    (n_H, n_W) = shape
    return np.ones(shape) * (dz / (n_H * n_W))

In [ ]:
def pool_backward(dA, cache, mode = "max"):
  (A_prev, hparameters) = cache
  stride = hparameters["stride"]
  f = hparameters["f"]
  
  m, n_H_prev, n_W_prev, n_C_prev = A_prev.shape
  m, n_H, n_W, n_C = dA.shape
  
  dA_prev = np.zeros(A_prev.shape)
  
  for i in range(m):
    a_prev = A_prev[i]
    for h in range(n_H):
      for w in range(n_W):
        for c in range(n_C):
          vert_start = h
          vert_end = vert_start + f
          horiz_start = w
          horiz_end = horiz_start + f
          
          if mode == "max":
            a_prev_slice = a_prev[vert_start:vert_end, horiz_start:horiz_end, c]
            mask = create_mask_from_window(a_prev_slice)
            dA_prev[i, vert_start:vert_end, horiz_start:horiz_end, c] += np.multiply(mask, dA[i, h, w, c])
              
          elif mode == "average":
            da = dA[i, h, w, c]
            shape = (f, f)
            dA_prev[i, vert_start:vert_end, horiz_start:horiz_end, c] += distribute_value(da, shape)
  
  return dA_prev

## **TENSORFLOW - Unet**

In [ ]:
################ https://keras.io/examples/vision/3D_image_classification/

import tensorflow as tf
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Dropout, Conv3DTranspose, concatenate

*Data Load/Split*

In [ ]:
## Load and Split
path = ''
image_path = os.path.join(path, './data/CameraRGB/')
mask_path = os.path.join(path, './data/CameraMask/')
image_list = os.listdir(image_path)
mask_list = os.listdir(mask_path)
image_list = [image_path+i for i in image_list]
mask_list = [mask_path+i for i in mask_list]

image_list_ds = tf.data.Dataset.list_files(image_list, shuffle=False)
mask_list_ds = tf.data.Dataset.list_files(mask_list, shuffle=False)

#for path in zip(image_list_ds.take(3), mask_list_ds.take(3)):
#    print(path)

image_filenames = tf.constant(image_list)
masks_filenames = tf.constant(mask_list)

dataset = tf.data.Dataset.from_tensor_slices((image_filenames, masks_filenames))

#for image, mask in dataset.take(1):
#    print(image)
#    print(mask)

In [ ]:
## Check image and labeled
N = 2
img = imageio.imread(image_list[N])
mask = imageio.imread(mask_list[N])
#mask = np.array([max(mask[i, j]) for i in range(mask.shape[0]) for j in range(mask.shape[1])]).reshape(img.shape[0], img.shape[1])

fig, arr = plt.subplots(1, 2, figsize=(14, 10))
arr[0].imshow(img)
arr[0].set_title('Image')
arr[1].imshow(mask[:, :, 0])
arr[1].set_title('Segmentation')

In [ ]:
## Process images

def process_path(image_path, mask_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=3)
    mask = tf.math.reduce_max(mask, axis=-1, keepdims=True)
    return img, mask

def preprocess(image, mask):
    input_image = tf.image.resize(image, (96, 128), method='nearest')
    input_mask = tf.image.resize(mask, (96, 128), method='nearest')

    return input_image, input_mask

image_ds = dataset.map(process_path)
processed_image_ds = image_ds.map(preprocess)

*U-net*

In [ ]:
def conv(inputs=None, n_filters=32, dropout_prob=0, max_pooling=True):

  conv = Conv3D(n_filters, # Number of filters
                3,   # Kernel size   
                activation='relu',
                padding='same',
                kernel_initializer='he_normal')(inputs)
  conv = Conv3D(n_filters, # Number of filters
                3,   # Kernel size
                activation='relu',
                padding='same',
                kernel_initializer='he_normal')(conv)

  if dropout_prob > 0:
    conv = Dropout(dropout_prob)(conv)

  if max_pooling:
    next_layer = MaxPooling3D((2,2,2))(conv)       
  else:
    next_layer = conv
      
  skip_connection = conv
  
  return next_layer, skip_connection

In [ ]:
def upsampling(expansive_input, contractive_input, n_filters=32):

  up = Conv3DTranspose(
                n_filters,
                3,    # Kernel size
                strides=2,
                padding='same')(expansive_input)
  merge = concatenate([up, contractive_input], axis=3)
  conv = Conv3D(n_filters,
                3,     # Kernel size
                activation='relu',
                padding='same',
                kernel_initializer='he_normal')(merge)
  conv = Conv3D(n_filters,
                3,   # Kernel size
                activation='relu',
                padding='same',
                kernel_initializer='he_normal')(conv)
  
  return conv

In [ ]:
######################## CHANGE INPUT SIZE ################################

def unet(input_size=(96, 128, 3), n_filters=32, n_classes=23):

  inputs = Input(input_size)

  cblock1 = conv(inputs, n_filters)

  cblock2 = conv(cblock1[0], 2*n_filters)
  cblock3 = conv(cblock2[0], 4*n_filters)
  cblock4 = conv(cblock3[0], 8*n_filters, dropout_prob=0.3) 
  cblock5 = conv(cblock4[0], 16*n_filters, dropout_prob=0.3, max_pooling=False) 

  ublock6 = upsampling(cblock5[0], cblock4[1],  n_filters * 8)
  ublock7 = upsampling(ublock6, cblock3[1],  n_filters * 4)
  ublock8 = upsampling(ublock7, cblock2[1],  n_filters * 2)
  ublock9 = upsampling(ublock8, cblock1[1],  n_filters)

  conv9 = Conv3D(n_filters,
                3,
                activation='relu',
                padding='same',
                kernel_initializer='he_normal')(ublock9)

  conv10 = Conv3D(n_classes, 1, padding='same')(conv9)

  model = tf.keras.Model(inputs=inputs, outputs=conv10)

  return model

*Main*

In [ ]:
######################## CHANGE INPUT SIZE ################################

img_height = 96
img_width = 128
num_channels = 3

model = unet((img_height, img_width, num_channels))
model.summary()

*Loss Function*

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

*Train*

In [ ]:
EPOCHS = 40
VAL_SUBSPLITS = 5
BUFFER_SIZE = 500
BATCH_SIZE = 32
processed_image_ds.batch(BATCH_SIZE)
train_dataset = processed_image_ds.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
print(processed_image_ds.element_spec)
model_history = unet.fit(train_dataset, epochs=EPOCHS)

*Predicted Masks*

In [ ]:
def create_mask(pred_mask):
  pred_mask = tf.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0]

*Model Accuracy*

In [ ]:
plt.plot(model_history.history["accuracy"])

*Example Results*